In [76]:
%%local
import os
username = os.environ['JUPYTERHUB_USER']
get_ipython().run_cell_magic('configure', line="-f", cell='{ "name":"%s-final-istaden", "executorMemory":"4G", "executorCores":4, "numExecutors":10, "driverMemory": "4G" }' % username)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6964,application_1618324153128_6616,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6915,application_1618324153128_6543,pyspark,busy,Link,Link,None,
6926,application_1618324153128_6556,pyspark,idle,Link,Link,None,
6939,application_1618324153128_6578,pyspark,idle,Link,Link,None,
6946,application_1618324153128_6589,pyspark,idle,Link,Link,None,
6947,application_1618324153128_6591,pyspark,idle,Link,Link,None,
6951,application_1618324153128_6600,pyspark,idle,Link,Link,None,
6952,application_1618324153128_6601,pyspark,idle,Link,Link,None,
6954,application_1618324153128_6603,pyspark,idle,Link,Link,None,
6955,application_1618324153128_6604,pyspark,idle,Link,Link,None,
6957,application_1618324153128_6606,pyspark,busy,Link,Link,None,


In [77]:
%%send_to_spark -i username -t str -n username

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'username' as 'username' to Spark kernel

In [78]:
#Loading the data 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
df = spark.read.orc('/data/sbb/orc/istdaten')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
df=df.selectExpr('betriebstag as date',
                 'produkt_id as transport_type',
                 'haltestellen_name as stop_name',
                 'ankunftszeit as arrival_scheduled',
                 'an_prognose as arrival_actual',
                 'durchfahrt_tf as stop_skip',
                 'bpuic as stop_id',
                  'linien_text as line_name',
                  'verkehrsmittel_text as line_type',
                  'linien_id as line_id'
                )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [92]:
df.select("stop_id").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

36708

In [89]:
stop_id_reachable.select("stop_id").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1597

In [93]:
import pyspark.sql.functions as F

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [94]:
#convert day of the trips between
df = df.withColumn('date', F.to_timestamp('date', "dd.MM.yyyy"))
df = df.withColumn('arrival_scheduled', F.to_timestamp('arrival_scheduled', "dd.MM.yyyy HH:mm"))
df = df.withColumn('arrival_actual', F.to_timestamp('arrival_actual', 'dd.MM.yyyy HH:mm:ss'))    

#keep only rows for stops that are not skipped
df=df.filter(df.stop_skip==False)

#keep only rows for stops during the week
df=df.withColumn("day_of_week",F.dayofweek(df.date))
df=df.filter(df.day_of_week.between(2,6))

#hours between 8am and 8pm
min_day_hour,max_day_hour=8,20
df = df.filter(F.hour(F.col('arrival_scheduled')).cast('int').between(min_day_hour, max_day_hour))

#only keep stops within the 15km radius
reachable_stops_path="/user/%s/final/parquet/reachable_stops" %username
stop_id_reachable=spark.read.parquet(reachable_stops_path)
stop_id_reachable=stop_id_reachable.withColumn("stop_id",F.split(stop_id_reachable.stop_id,':')[0]).drop("stop_name")

df=df.join(stop_id_reachable,on="stop_id")

#add hour column
df=df.withColumn("hour",F.hour(F.col("arrival_scheduled")))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [95]:
df.select("stop_id").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1530

In [97]:
#filter the rows where transport_type is null
df=df.where((F.col("transport_type")=="Tram") | (F.col("transport_type")=="Zug")|(F.col("transport_type")=="Bus"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
stop_id_reachable.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------------+----------------+-------------+--------------+------------------+
|stop_id|        stop_lat|        stop_lon|location_type|parent_station|distance_zurich_HB|
+-------+----------------+----------------+-------------+--------------+------------------+
|8502276|47.3621754906935|8.36679177646695|            1|              |13.179043039878453|
|8502276|47.3622971874385|8.36679177646695|         null|      8502276P|13.177207987631474|
|8502276|47.3622363391011|8.36679177646695|         null|      8502276P|13.178123808381171|
|8502276|47.3621754906935|8.36679177646695|         null|      8502276P|13.179043039878453|
|8503879|47.3724396243058|8.35652403276804|         null|              |13.844981913831946|
|8576183|47.3427977889857|8.56732269736887|         null|              | 4.433088316413242|
|8591406|47.3961605624575|8.54482888265138|         null|              |2.0299214545979867|
|8591265|47.3890818922336|8.49469390663769|         null|              | 3.63376

In [98]:
#computing actual delays
@F.udf
def relu(x):
    return max(x,0)
df=df.withColumn("delay",relu((F.col("arrival_actual").cast("long")-F.col("arrival_scheduled").cast("long"))))
df = df.where(~(F.col('delay').isNull()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#statistics type of transport average delays
transport_avg_delays=df.groupby("transport_type").agg(F.mean('delay'))
transport_avg_delays.show()

In [ ]:
transport_avg_delays=transport_avg_delays.withColumnRenamed("avg(delay)","avg_delay")

In [ ]:
transport_avg_delays.show()

In [ ]:
import matplotlib
matplotlib.use('agg')
import matplotlib.pylab as plt

plt.rcParams['figure.figsize'] = (30,8)
plt.rcParams['font.size'] = 12
plt.style.use('fivethirtyeight')

In [ ]:
import matplotlib.pyplot as plt
# Plot transport average delays

t_avg_delays_plot=transport_avg_delays.toPandas()

plt.bar(x=t_avg_delays_plot.transport_type,height=t_avg_delays_plot.avg_delay)
#transport_avg_delays["avg_delay"].plot.bar
plt.xlabel('average delay distribution by train')
plt.ylabel('average delay (seconds)')
plt.xticks(range(len(t_avg_delays_plot)), t_avg_delays_plot.transport_type)

In [ ]:
%matplot plt

In [ ]:
hour_avg_delays=df.groupby("hour").agg(F.mean('delay'))

hour_avg_delays=hour_avg_delays.withColumnRenamed("avg(delay)","avg_delay")

In [ ]:
# Plot transport average delays
plt.cla()
h_avg_delays_plot=hour_avg_delays.toPandas().sort_values(by="hour").reset_index(drop=True)

plt.bar(x=h_avg_delays_plot.hour, height=h_avg_delays_plot.avg_delay, color=(0.2, 0.4, 0.6, 0.6))
 
plt.xlabel('average delay distribution by hour')
plt.ylabel('average delay (seconds)')
plt.xticks(h_avg_delays_plot.hour)
# Show the graph
%matplot plt

In [ ]:
h_avg_delays_plot

In [105]:
train_types=df.where(F.col("transport_type")=="Zug").groupby("line_name").agg(F.mean('delay'))
#train_types.filter(train_types.line_name[0:2]=="RE").show()
train_types.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------------------+
|line_name|        avg(delay)|
+---------+------------------+
|     IR13| 65.32800626622087|
|       IC| 120.4030260030296|
|      ZUG|             362.0|
|       RE| 48.46594256661397|
|       NJ| 532.7103941633504|
|      TGV| 371.7994513867723|
|     IR16|  84.6015362190911|
|      S10| 62.09292183598789|
|      IC5|  82.2860826042917|
|       S6| 61.67543553967913|
|       S8|61.763740268928316|
|    ZUG46|               0.0|
|       S7| 53.77962510252617|
|    ZUG36|             167.0|
|      S18|               0.0|
|       S4| 90.93178599027974|
|      S19| 52.28142776406349|
|      S23| 81.26838091013019|
|       S3| 58.52042471203553|
|      S16|   44.422459280037|
+---------+------------------+
only showing top 20 rows

In [62]:
df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[stop_id: string, date: timestamp, transport_type: string, stop_name: string, arrival_scheduled: timestamp, arrival_actual: timestamp, stop_skip: string, line_name: string, line_type: string, line_id: string, day_of_week: int, stop_name: string, stop_lat: double, stop_lon: double, location_type: int, parent_station: string, true_stop_id: string, distance_zurich_HB: double, hour: int, delay: string]

In [106]:
train_types_stops=df.where(F.col("transport_type")=="Zug").groupby("line_name", "hour", "stop_id").agg(F.mean('delay'))
#train_types.filter(train_types.line_name[0:2]=="RE").show()
train_types_stops.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----+-------+------------------+
|line_name|hour|stop_id|        avg(delay)|
+---------+----+-------+------------------+
|      S18|   9|8530813|               0.0|
|      S10|  15|8503052|54.899648156646776|
|       S4|  16|8503087|62.882486979166664|
|       S4|  17|8503088| 96.81582446808511|
|      S17|  15|8502268|56.642736486486484|
|       S3|  14|8503020|38.809453471196456|
|       S3|  14|8503305| 39.92926670992862|
|       S3|  20|8503020| 38.55282555282555|
|       S6|   8|8503526| 99.38361929426999|
|       S6|  16|8503007| 99.81791720569211|
|       S6|  19|8503111| 90.99481865284974|
|       S8|   8|8503204|106.08058252427185|
|       S8|  11|8503000|22.001940491591203|
|      S12|  15|8503147| 32.74444444444445|
|      S12|  19|8503147|41.098381466201204|
|      S21|  18|8503008| 79.24966078697422|
|        S|  17|8503306|            35.375|
|      S15|  18|8503147| 160.9047619047619|
|      EXT|  19|8503103|             105.0|
|       S7|  19|8503103|        

In [107]:
train_types_hour=df.where(F.col("transport_type")=="Zug").groupby("line_name","hour").agg(F.mean('delay'))
train_types_hour.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----+------------------+
|line_name|hour|        avg(delay)|
+---------+----+------------------+
|      ICE|  13| 75.14521841794569|
|       IC|  13|102.72360062978446|
|     IR75|  18| 90.20963406673579|
|     IR37|  12| 64.38737854859639|
|     IR37|  15| 70.57067383739323|
|     IR13|  17| 72.65537099903007|
|      S15|  15| 53.14881894301051|
|      S21|   8|30.722813744285503|
|     IR35|  16| 37.01428571428571|
|       RE|  20| 49.79948075924537|
|       AG|   8|            1814.0|
|     IR16|  10|              58.0|
|       EC|  14|206.72096641347122|
|       S7|  11| 40.29147569201209|
|       S7|  12| 43.45201537045191|
|      EXT|  12|112.08719346049047|
|       S8|  12| 46.33350075899634|
|       S9|   8| 77.01190748981914|
|      S11|  16|46.192433632759574|
|      S25|  19| 92.43643441411254|
+---------+----+------------------+
only showing top 20 rows

In [108]:
train_types_hour.groupby('line_name').count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-----+
|line_name|count|
+---------+-----+
|     IR13|   13|
|       IC|   13|
|      ZUG|    1|
|       RE|   13|
|       NJ|    5|
|      TGV|    5|
|     IR16|   13|
|      S10|   13|
|      IC5|   13|
|       S6|   13|
|       S8|   13|
|    ZUG46|    1|
|       S7|   13|
|    ZUG36|    1|
|      S18|   13|
|       S4|   13|
|      S19|   13|
|      S23|    4|
|       S3|   13|
|      S16|   13|
+---------+-----+
only showing top 20 rows

In [109]:
bus_types=df.where(F.col("transport_type")=="Bus").groupby("line_name").agg(F.mean('delay'))
bus_types.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------------------+
|line_name|        avg(delay)|
+---------+------------------+
|      451| 82.52002079033552|
|      919| 66.11488035637656|
|      307|107.98843247350868|
|      205| 50.28392293487852|
|      743| 86.86185857969186|
|      155| 42.41823187519119|
|      132|    24.80567838786|
|      727| 81.16999332264646|
|      200|116.61423365078583|
|      521|   74.595160085965|
|      961| 137.5145813802553|
|       69| 67.50659419637475|
|      309| 99.29527676178792|
|      703|121.41838113200617|
|      974| 79.19863145502705|
|       73|  74.3761608040201|
|       64| 62.32576592517063|
|      308| 94.99145552397123|
|      652|  90.1991169152099|
|      910| 95.08755089755076|
+---------+------------------+
only showing top 20 rows

In [110]:
bus_types_stops=df.where(F.col("transport_type")=="Bus").groupby("line_name", "hour", "stop_id").agg(F.mean('delay'))
bus_types_stops.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----+-------+------------------+
|line_name|hour|stop_id|        avg(delay)|
+---------+----+-------+------------------+
|      449|  16|8576260|30.089437474706596|
|      450|   9|8576235| 34.91301504251145|
|      451|  11|8576249| 99.08630527817404|
|      452|  10|8590760| 59.49832214765101|
|      452|  16|8590878| 27.57546494992847|
|      485|  15|8591402| 42.18181818181818|
|      640|  13|8594339| 95.00698757763975|
|      640|  12|8503382| 62.17404580152672|
|      650|  18|8503374|62.187721788502486|
|      655|  10|8503381| 82.57342115459124|
|      658|   9|8576169| 138.1376673040153|
|      725|  15|8503156| 77.93617577818468|
|      725|  12|8587859| 90.67864271457086|
|      726|   9|8503154|207.47377809489834|
|      726|  15|8576127| 71.44610992148465|
|      727|  13|8590640| 53.85274869109948|
|      732|  19|8590600| 68.69421487603306|
|      733|  17|8587420| 84.28103170338528|
|      733|  13|8580437| 68.16503496503496|
|      733|  14|8576153| 65.4094

In [111]:
bus_types_hour=df.where(F.col("transport_type")=="Bus").groupby("line_name", "hour").agg(F.mean('delay'))
bus_types_hour.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----+------------------+
|line_name|hour|        avg(delay)|
+---------+----+------------------+
|      134|  12|43.472376265076946|
|      155|   8|57.291875284478834|
|      811|  18|101.79324439212608|
|      842|  17| 134.4118087335871|
|      973|  17| 86.01799626906617|
|       32|  18| 104.4351895192534|
|       40|  14| 93.80258306888183|
|       75|  19| 80.64258898169436|
|      751|  18| 88.03139547472124|
|      652|  19|61.741764492940995|
|      754|  17| 157.4464488783211|
|      650|   9|161.63326334542217|
|      245|  11| 69.17903682719546|
|      795|  18| 82.79540081667741|
|      245|  19|  46.1589215740383|
|      225|  15| 66.32331893494938|
|      227|  15| 80.71390102631847|
|      521|   8| 94.44635286863134|
|      521|   9| 66.13880161579893|
|      504|  13| 86.26912846189954|
+---------+----+------------------+
only showing top 20 rows

In [112]:
tram_types=df.where(F.col("transport_type")=="Tram").groupby("line_name").agg(F.mean('delay'))
tram_types.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------------------+
|line_name|        avg(delay)|
+---------+------------------+
|        7| 70.81191261690066|
|       15| 75.00601441653484|
|       11| 66.86055264518396|
|        3| 59.33061610213354|
|        8| 70.40891044996407|
|        E| 90.85397553516819|
|        5| 59.46522312547608|
|       17|  68.8109536244329|
|        6| 56.65332733859282|
|       8E|   71.322913028292|
|        9|57.419847911867606|
|       10| 70.04290109313014|
|        4| 69.85353917808457|
|       12| 51.71743727892853|
|       13| 69.13658160251748|
|       14|57.614791397629645|
|        2| 54.24695457437215|
+---------+------------------+

In [113]:
tram_types_stops=df.where(F.col("transport_type")=="Tram").groupby("line_name", "hour", "stop_id").agg(F.mean('delay'))
tram_types_stops.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----+-------+------------------+
|line_name|hour|stop_id|        avg(delay)|
+---------+----+-------+------------------+
|       13|  20|8591067| 69.50646153846154|
|       14|  11|8591168|  55.8284845496384|
|       15|  11|8591183| 68.39863627111744|
|       17|  16|8591131|50.018172241243086|
|       17|  18|8591427| 72.19849246231156|
|       13|  15|8591306| 63.11587982832618|
|        2|   8|8590318| 72.01213538187486|
|        2|   8|8591299| 57.48425948999086|
|        2|  19|8591448| 52.09071348653068|
|        4|  17|8591309| 85.45589570096543|
|        4|   9|8591178|167.36363636363637|
|        5|   9|8591220|47.534160037435655|
|        5|  13|8591412| 45.73444563162653|
|        6|  10|8591074|54.714146103102856|
|        7|  13|8591283| 56.16522882261113|
|        8|  12|8530813|  62.3380490212206|
|        8|  17|8591384|  84.5840786399162|
|        9|  20|8591343| 64.70736369680851|
|        9|  11|8591390| 52.19317338609943|
|       10|  18|8588553|142.5216

In [114]:
tram_types_hour=df.where(F.col("transport_type")=="Tram").groupby("line_name", "hour").agg(F.mean('delay'))
tram_types_hour.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----+------------------+
|line_name|hour|        avg(delay)|
+---------+----+------------------+
|       14|  10| 55.37009190703116|
|        5|  15| 66.51267330325216|
|        8|  18| 88.76431724137932|
|        7|  16| 78.86462999440376|
|        8|  19| 68.62609447355867|
|       14|   9| 51.88008392200643|
|       15|  11| 69.42126711721973|
|        3|  13| 57.00602907371504|
|       13|  19| 68.37475685234305|
|       8E|   9| 69.49776785714286|
|        9|  18|47.478068418624225|
|        9|   8| 58.06513618497229|
|       14|  18|62.205553500097196|
|       17|  19| 62.22989701445959|
|        3|   8|62.483166967097674|
|       15|  14| 75.62902567937323|
|        2|  20| 54.93717576796998|
|        3|  19| 54.46267692001724|
|       8E|  10| 80.52232142857143|
|       12|  20| 51.64720644234971|
+---------+----+------------------+
only showing top 20 rows

In [115]:
#Renaming
train_types = train_types.withColumnRenamed("avg(delay)","avg_delay")
train_types_hour = train_types_hour.withColumnRenamed("avg(delay)","avg_delay")
train_types_stops = train_types_stops.withColumnRenamed("avg(delay)","avg_delay")

bus_types= bus_types.withColumnRenamed("avg(delay)","avg_delay")
bus_types_hour= bus_types_hour.withColumnRenamed("avg(delay)","avg_delay")
bus_types_stops = bus_types_stops.withColumnRenamed("avg(delay)","avg_delay")

tram_types = tram_types.withColumnRenamed("avg(delay)","avg_delay")
tram_types_hour = tram_types_hour.withColumnRenamed("avg(delay)","avg_delay")
tram_types_stops = tram_types_stops.withColumnRenamed("avg(delay)","avg_delay")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
#transport_avg_delays.write.parquet('/user/%s/final/parquet/transport_avg_delays' % username)
#hour_avg_delays.write.parquet('/user/%s/final/parquet/hour_avg_delays' % username)

train_types.write.mode('overwrite').parquet('/user/%s/final/parquet/train_types' % username)
bus_types.write.mode('overwrite').parquet('/user/%s/final/parquet/bus_types' % username)
tram_types.write.mode('overwrite').parquet('/user/%s/final/parquet/tram_types' % username)

train_types_hour.write.mode('overwrite').parquet('/user/%s/final/parquet/train_types_hour' % username)
bus_types_hour.write.mode('overwrite').parquet('/user/%s/final/parquet/bus_types_hour' % username)
tram_types_hour.write.mode('overwrite').parquet('/user/%s/final/parquet/tram_types_hour' % username)

train_types_stops.write.parquet('/user/%s/final/parquet/train_types_stops' % username)
bus_types_stops.write.parquet('/user/%s/final/parquet/bus_types_stops' % username)
tram_types_stops.write.parquet('/user/%s/final/parquet/tram_types_stops' % username)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
default_delay = df.select(F.mean(F.col('delay')).alias('mean_delay'),).collect()


In [ ]:
default_delay.show()